<a href="https://colab.research.google.com/github/NasrSiala/Detection___Tracking/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing ffmpeg
!apt-get -qq install ffmpeg

# import the library
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display, HTML
from base64 import b64encode

%matplotlib inline

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving City Traffic.mp4 to City Traffic.mp4
Saving coco_class_labels.txt to coco_class_labels.txt
Saving frozen_inference_graph.pb to frozen_inference_graph.pb
Saving ssd_mobilenet_v2_coco_2018_03_29.pbtxt to ssd_mobilenet_v2_coco_2018_03_29.pbtxt


In [ ]:
# Read file
classFile  = "coco_class_labels.txt"
with open(classFile) as fp:
    labels = fp.read().split("\n")

In [ ]:
modelFile  = "frozen_inference_graph.pb"
configFile = "ssd_mobilenet_v2_coco_2018_03_29.pbtxt"

In [ ]:
# Read the Tensorflow network
net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

In [ ]:
# For each file in the directory
def detect_objects(net, im, dim = 300):

    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(im, 1.0, size=(dim, dim), mean=(0, 0, 0), swapRB=True, crop=False)

    # Pass blob to the network
    net.setInput(blob)

    # Peform Prediction
    objects = net.forward()
    return objects

In [ ]:
FONTFACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
THICKNESS = 1

In [ ]:
def display_text(im, text, x, y):
    # Get text size
    textSize = cv2.getTextSize(text, FONTFACE, FONT_SCALE, THICKNESS)
    dim = textSize[0]
    baseline = textSize[1]

    # Use text size to create a black rectangle
    cv2.rectangle(
        im,
        (x, y - dim[1] - baseline),
        (x + dim[0], y + baseline),
        (0, 0, 0),
        cv2.FILLED,
    )

    # Display text inside the rectangle
    cv2.putText(
        im,
        text,
        (x, y - 5),
        FONTFACE,
        FONT_SCALE,
        (0, 255, 255),
        THICKNESS,
        cv2.LINE_AA,
    )

In [ ]:
def display_objects(im, objects, threshold=0.25):
    rows = im.shape[0]
    cols = im.shape[1]

    # For every Detected Object
    for i in range(objects.shape[2]):
        # Find the class and confidence
        classId = int(objects[0, 0, i, 1])
        score = float(objects[0, 0, i, 2])

        # Recover original coordinates from normalized coordinates
        x = int(objects[0, 0, i, 3] * cols)
        y = int(objects[0, 0, i, 4] * rows)
        w = int(objects[0, 0, i, 5] * cols - x)
        h = int(objects[0, 0, i, 6] * rows - y)

        # Check if the detection is of good quality
        if score > threshold:
            display_text(im, "{}".format(labels[classId]), x, y)
            cv2.rectangle(im, (x, y), (x + w, y + h), (255, 255, 255), 2)

    return im

In [ ]:
# Read video
source = 'City Traffic.mp4'
cap = cv2.VideoCapture(source)

In [ ]:
# Default resolutions of the frame are obtained.
# Convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.
out_mp4 = cv2.VideoWriter("Out.mp4", cv2.VideoWriter_fourcc(*"XVID"), 100, (frame_width, frame_height))

In [ ]:
# Read until video is completed
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()

    if ret:
      objects = detect_objects(net, frame)
      frame = display_objects(frame, objects, 0.2)
      # Write the frame to the output files
      out_mp4.write(frame)

    # Break the loop
    else:
        break

In [ ]:
# When everything done, release the VideoCapture and VideoWriter objects
cap.release()
out_mp4.release()

In [ ]:
# Change video encoding of mp4 file from XVID to h264
!ffmpeg -y -i "/content/Out.mp4" -c:v libx264 "Out_x264.mp4"  -hide_banner -loglevel error

In [ ]:
#CREATE FINAL FILE
mp4 = open("/content/Out.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""<video width=700 controls><source src="{data_url}" type="video/mp4"></video>""")

Buffered data was truncated after reaching the output size limit.